In [1]:
from MotionGPT import *

In [2]:
loader = FBXDataLoader("FBXDataStand")

In [3]:
loader.LoadData()

FBXDataLoader loading data...


In [4]:
len(loader.raw_data)

149

In [5]:
len(loader.raw_data[0])

46

In [6]:
loader.PrepareTrainingData(12)

In [7]:
len(loader.train_data)

1411

In [8]:
len(loader.test_data)

353

In [9]:
loader.test_data[0]

[[-0.06213372137099814,
  -0.056723200689815706,
  -0.01972222054753592,
  -0.0029670597283903604,
  0.0020943951023931952,
  0.00715584993317675,
  -0.0005235987755982988,
  0.003839724354387525,
  0.011693705988362007,
  -0.0005235987755982988,
  0.003839724354387525,
  0.011693705988362007,
  0.0027925268031909274,
  0.0033161255787892262,
  0.0029670597283903604,
  0.08325220532012952,
  -0.046076692252650306,
  -0.1499237827463129,
  1.3480923142404202,
  0.045553093477052,
  0.5510004448546099,
  0.03630284844148206,
  -0.00715584993317675,
  0.21868975527488949,
  0.05201081170943102,
  -0.39304814754912304,
  -0.22549653935766736,
  0.060039326268604934,
  0.027401669256310976,
  0.10786134777324957,
  1.324355836413297,
  0.04014257279586957,
  -0.29443704481144345,
  0.03769911184307752,
  0.009075712110370514,
  -0.2698279023583233,
  0.10349802464326374,
  -0.23300145514124296,
  0.10192722831646883,
  -0.010821041362364843,
  -0.25394540616517497,
  0.12269664641520138,
  

In [ ]:
len(loader.train_data[180])

In [ ]:
from VRNN import *

In [ ]:
input_dim = len(loader.train_data[0][1])
h_dim = 64
z_dim = 10
n_layers = 2
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  #device

In [ ]:
print(device)

In [ ]:
model = VRNN(input_dim, h_dim, z_dim, n_layers, device)
model.to(device)

In [ ]:
batch_size = 16
n_epochs = 100
clip = 10
learning_rate = 1e-3
print_every = 10

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
for epoch in range(n_epochs):
    batch_idx = 0   
    for batch_data, pad_data in loader.next_batch(batch_size):
        batch_idx += 1

        batch_data = batch_data.to(device)
        pad_data = pad_data.to(device)

        optimizer.zero_grad()

        kld_loss, nll_loss, _, _ = model(batch_data, pad_data)
        loss = kld_loss + nll_loss
        loss.backward()

        #grad norm clipping, only in pytorch version >= 1.10
        torch.nn.utils.clip_grad_norm(model.parameters(), clip)
        optimizer.step()

        #printing
        if batch_idx % print_every == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\t KLD Loss: {:.6f} \t NLL Loss: {:.6f}'.format(
                epoch, batch_idx * len(batch_data), len(loader.train_data),
                100. * batch_idx / len(loader.train_data),
                kld_loss.data / batch_size,
                nll_loss.data / batch_size))

In [ ]:
model.sample(20)

-------------------------------------

In [ ]:
from MotionGPT import *

In [ ]:
maker = FBXDataMaker(12345)

In [ ]:
maker.mc.UndoToBeginning(10000)

In [ ]:
maker.GetModelNameSpace()
print(maker.namespace)

In [ ]:
maker.LoadModel("runs/2020_08_19_14_34_04_nr.pth")

In [ ]:
maker.SampleAnim(20)

In [ ]:
#loader 
loader = FBXDataLoader("FBXData", radian=True)
loader.LoadData()
loader.PrepareTrainingData(12)

In [ ]:
train_sample = torch.FloatTensor(loader.train_data[np.random.randint(1000)])

In [ ]:
train_sample = train_sample.unsqueeze(1)

In [ ]:
train_sample.shape

In [ ]:
a, b = next(iter(loader.next_batch()))

In [ ]:
a.shape, b.shape

In [ ]:
train_sample_mask = torch.LongTensor([[1]]*train_sample.size(0))

In [ ]:
train_sample_mask.shape

In [ ]:
train_sample = train_sample.to(maker.model.device)
train_sample_mask = train_sample_mask.to(maker.model.device)


In [ ]:
result = maker.model.forward(train_sample, train_sample_mask)

In [ ]:
i = 0

In [ ]:
for i in range(len(train_sample)):
    maker.GenerateMayaPose(result[3][0][i][0], i * frame_gap)

In [ ]:
result[3][0][1]

In [ ]:
maker_ori = FBXDataMaker(12346)

In [ ]:
maker_ori.GetModelNameSpace()
print(maker_ori.namespace)

In [ ]:
maker_ori.mc.UndoToBeginning(10000)

In [ ]:
for i in range(len(train_sample)):
    maker_ori.GenerateMayaPose(train_sample[i][0], i * frame_gap)

In [ ]:
np.sqrt(3000/63)